In [9]:
from transformers import AutoModel, AutoProcessor , BarkModel
import torch
import numpy as np

processor = AutoProcessor.from_pretrained("suno/bark-small")
# model = AutoModel.from_pretrained("suno/bark-small")


device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = BarkModel.from_pretrained("suno/bark-small", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)
model =  model.to_bettertransformer()


voice_preset = "v2/en_speaker_9"

inputs = processor("Hello, my name is Aditya. And, uh — and I like pizza. [laughs] But I also have other interests such as playing tic tac toe.", voice_preset=voice_preset).to(device)
inputs['attention_mask'] = inputs['input_ids'].ne(processor.tokenizer.pad_token_id).long().to(device)
audio_array = model.generate(**inputs)
audio_array = audio_array.cpu().numpy().squeeze()
import scipy

sample_rate = model.generation_config.sample_rate
scipy.io.wavfile.write("bark_out.wav", rate=sample_rate, data=audio_array.astype(np.float32))


c:\Users\arya2\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\Users\arya2\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\encodec\modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview f

In [7]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

device = "cuda:0" if torch.cuda.is_available() else "cpu"
attn_implementation = "flash_attention_2" 
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-mini-v1",attn_implementation=attn_implementation,torch_dtype=torch.float16).to(device)


tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-mini-v1")



In [8]:
prompt = "Hello, my name is Suno. And, uh — and I like pizza. [laughs] But I also have other interests such as playing tic tac toe."
description = "A female speaker delivers a slightly expressive and animated speech with a moderate speed and pitch. The recording is of very high quality, with the speaker's voice sounding clear and very close up."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze().todtype(np.float32)
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)

AttributeError: 'numpy.ndarray' object has no attribute 'todtype'

In [ ]:
import ollama
PODLM = ollama.Client()
print("Generating podcast...")
podcast = PODLM.generate(model="PODLM4" ,prompt='''AI based SWOC analysis of student using academic progress
Vishwakarma Institute of Information Technology
Course: TY Project
Guide: Prof. Priya Shelke
Abstract
It is time consuming and difficult to analyze your weaknesses and come up with a good solution.
At the same time some students do not know the concrete areas where they should improve themselves
since the number of academic jobs required them time to finish their assignments. Furthermore, without
explicit guidance from the teacher, students cannot figure out the hierarchy of improvement or which source
is good or not. Furthermore, students may have difficulty noticing their own long-term growth when they
have no benchmarks against which to measure their efforts - it may appear that their efforts aren't yielding
the results they want. Without opportunities for self-reflection or personal feedback, students may be left
wondering what they should do next, how to overcome their current difficulties, and so on.
This project will focus on achieving a general web application that will enable students to selfassess their studies and get insights via a SWOC (Strengths, Weaknesses, Opportunities and Challenges)
analysis. Through a machine learning algorithm, the analysis of the students' performance shall be
conducted based on different parameters such as examinations performance, participation in competitions,
project success rate among others and a grade will be given to the students respective to their performance.
SWOC analysis will give specific, personalized recommendations to help students learn where they excel
and where they struggle. Through this analysis, students will be able to allocate their efforts more wisely.
The platform will also provide continuous progress reports in the form of dynamic visualizations of
academic progress over time. Further, the application will include a chatbot system for the students to ask
questions regarding improvements and different pathways which they can follow to achieve their dreams.''')
print(podcast)

Generating podcast...
